In [2]:
!pip install librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 3.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.7 MB/s eta 0:00:00:00:0100:01


In [53]:
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np
import torch

In [4]:
audio_path ='data/gwilliams2022/download/stimuli/audio/lw1_0.wav'

In [5]:
speech, sr = torchaudio.load(audio_path)
speech = speech[0].numpy().squeeze()

In [6]:
sr

22050

In [8]:
speech.shape

(2114191,)

In [11]:
speech = librosa.resample(np.asarray(speech), orig_sr=sr, target_sr=16_000)
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

In [13]:
speech.shape

(1534107,)

In [14]:
model_name = "facebook/wav2vec2-large-xlsr-53"

In [16]:
from transformers import Wav2Vec2Model
from transformers import Wav2Vec2Model, Wav2Vec2Config
from transformers import Wav2Vec2FeatureExtractor

In [17]:
config = Wav2Vec2Config.from_pretrained(model_name)

In [18]:
config

Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.1,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.0,
  "gradient_checkpointing": false,
  "hi

In [19]:
model =Wav2Vec2Model(config)

In [20]:
model

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (1-4): 4 x Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=1024, bias=True)
    (dropout)

In [21]:
feature_extractor =Wav2Vec2FeatureExtractor.from_pretrained(model_name)

In [22]:
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [23]:
out = feature_extractor(speech,return_tensors="pt",
                                     sampling_rate=16_000,
                                     do_normalize=True).input_values

In [24]:
out.shape

torch.Size([1, 1534107])

In [25]:
speech.shape

(1534107,)

In [40]:
res =model(out, output_hidden_states=True)

In [56]:
layers =[14, 15, 16, 17, 18]

In [48]:
out= res.get('hidden_states')

In [54]:
out = torch.stack(out)

In [60]:
out[layers].mean(0).shape

torch.Size([1, 4793, 1024])

In [30]:
res.keys()

odict_keys(['last_hidden_state', 'extract_features'])

In [32]:
res.last_hidden_state.shape

torch.Size([1, 4793, 1024])

In [35]:
4793

4793

In [34]:
1534107 /16000

95.8816875

In [37]:
res.extract_features.shape

torch.Size([1, 4793, 512])

In [38]:
4793 /95

50.45263157894737

In [39]:
50 *  3

150

In [63]:
3*16000/320

150.0

In [62]:
1534107/4793

320.0723972459837